In [ ]:
import csv
import itertools
import pathlib
import shutil
from math import ceil, sqrt
from pathlib import Path
from pprint import pprint
from typing import Callable, List, Optional, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import rootutils
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from skimage.metrics import structural_similarity
from sklearn.model_selection import GroupShuffleSplit
from torch import Tensor
from torch.utils.data import ConcatDataset, Dataset
from torchmetrics.classification.accuracy import Accuracy
from torchvision.io import read_image
from tqdm.notebook import tqdm

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png')

root = rootutils.setup_root(search_from=".", indicator=".project-root", pythonpath=True)

from src.data.components.dataset import (
    FetalBrainPlanesDataset,
    USVideosDataset,
    USVideosFrameDataset,
    VideoQualityDataset,
)
from src.data.components.transforms import (
    Affine,
    HorizontalFlip,
    LabelEncoder,
    RandomPercentCrop,
    VerticalFlip,
)
from src.data.utils.utils import show_numpy_images, show_pytorch_images
from src.models.fetal_module import FetalLitModule
from src.models.quality_module import QualityLitModule

data_dir = root / "data"
root

In [ ]:
model_log_dir = root / "logs" / "train" / "multiruns" / "2023-11-07_21-10-40" / "0"  # frosty-forest-2691

checkpoint = sorted(model_log_dir.glob("checkpoints/epoch_*.ckpt"))[-1]
model = FetalLitModule.load_from_checkpoint(str(checkpoint))
# disable randomness, dropout, etc...
model.eval()

model.hparams.net_spec.name

# Plot Videos Probabilities

In [ ]:
def init_counts():
    counts = {}
    for min_probability in min_probabilities:
        count = {}
        for label in label_names:
            count[label] = 0
        counts[min_probability] = count
    return counts


def label_videos():
    selected_path = video_dataset_dir / "selected"
    videos = list(selected_path.iterdir())
    for i, frames_path in enumerate(tqdm(videos, desc="Label videos")):
        label_video(frames_path)


def label_video(frames_path: Path):
    frames_paths = list(frames_path.iterdir())
    epochs = ceil(len(frames_paths) / batch_size)
    for i in range(epochs):
        frames = frames_paths[(i * batch_size) : ((i + 1) * batch_size)]
        label_frames(frames)


def label_frames(frames):
    with torch.no_grad():
        frames = get_frames_tensor(frames)
        frames = frames.to(model.device)

        results = [model(t(frames))[1] for t in transforms]
        logits = torch.mean(torch.stack(results, dim=1), dim=1)

        y_hats = F.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)
        count_labels(y_hats, preds)


def get_frames_tensor(frame_paths):
    frames = []
    for frame_path in frame_paths:
        frame = cv2.imread(str(frame_path))
        frame = PIL.Image.fromarray(frame)
        frame = TF.to_tensor(frame)
        frames.append(frame)
    return torch.stack(frames, dim=0)


def count_labels(y_hats, preds):
    for y_hat, pred in zip(y_hats, preds):
        for min_probability in min_probabilities:
            if y_hat[pred] > min_probability:
                counts[min_probability][label_names[pred]] += 1


video_dataset_dir = data_dir / "US_VIDEOS_0.3"
label_names = FetalBrainPlanesDataset.labels
min_probabilities = [0.0, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 0.995]
counts = init_counts()

horizontal_flips = [False]
vertical_flips = [False]
rotate_degrees = [0]
translates = [(0.0, 0.0)]
scales = [1.0]

transforms = [
    torch.nn.Sequential(
        T.Grayscale(),
        T.Resize((165, 240), antialias=False),
        HorizontalFlip(flip=horizontal_flip),
        VerticalFlip(flip=vertical_flip),
        Affine(degrees=rotate_degree, translate=translate, scale=scale),
        T.ConvertImageDtype(torch.float32),
    )
    for horizontal_flip, vertical_flip, rotate_degree, translate, scale in itertools.product(
        horizontal_flips,
        vertical_flips,
        rotate_degrees,
        translates,
        scales,
    )
]
batch_size = 32

# model.to("cpu")
model.to("cuda")

label_videos()

In [ ]:
def plot_videos_probabilities(counts):
    with plt.style.context("seaborn-v0_8-muted"):
        fig, ax = plt.subplots(figsize=(15, 8))

    for i, (min_prob, count) in enumerate(counts.items()):
        labels = list(count.keys())
        values = list(count.values())
        ax.bar(labels, values, label=str(min_prob))

    ax.legend()
    ax.set_title("Probabilities on video dataset")


plot_videos_probabilities(counts)
plt.show()
plt.savefig(str(model_log_dir / "plot_videos_probabilities.pdf"), bbox_inches="tight")

# Test-Time Augmentation

In [ ]:
label_names = FetalBrainPlanesDataset.labels

test = FetalBrainPlanesDataset(
    data_dir=data_dir,
    train=False,
    transform=torch.nn.Sequential(
        T.Grayscale(),
        T.Resize((165, 240), antialias=False),
    ),
    target_transform=LabelEncoder(labels=label_names),
)

In [ ]:
def evaluate():
    epochs = ceil(len(test) / batch_size)

    logits_batches = []
    y_batches = []

    for epoch_idx in tqdm(range(epochs), desc="Evaluate"):
        batch = [test[i] for i in range(epoch_idx * batch_size, (epoch_idx + 1) * batch_size) if i < len(test)]
        x = torch.stack([x for x, _ in batch])
        y = torch.stack([y for _, y in batch])

        x = x.to(model.device)
        logits_batch = [evaluate_x(x, tran) for tran in tqdm(transforms, desc="Transforms", leave=False)]
        logits_batch = torch.stack(logits_batch, dim=1)

        logits_batches.append(logits_batch)
        y_batches.append(y)

    return torch.cat(logits_batches, dim=0), torch.cat(y_batches, dim=0)


def evaluate_x(x, transform):
    with torch.no_grad():
        x = transform(x)
        _, logits = model(x)
        return logits.cpu()


# horizontal_flips=[False]
horizontal_flips = [False, True]

vertical_flips = [False, True]

# rotate_degrees=[0]
# rotate_degrees=[0, -5, 5]
rotate_degrees = [0, -5, -10, 5, 10]
# rotate_degrees = [0, -5, -10, -15, 5, 10, 15]

# translates=[(0.0, 0.0)]
translates = [(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]
# translates = list(itertools.product([0.0, 0.1, -0.1], [0.0, 0.1, -0.1]))

# scales=[1.0]
# scales=[1.0, 1.05, 1.10]
# scales=[1.0, 1.05, 1.10, 1.15]
scales = [1.0, 1.05, 1.10, 1.15, 1.20]
# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25]
# scales = [1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30]

params = [
    {
        "horizontal_flip": horizontal_flip,
        "vertical_flips": vertical_flips,
        "rotate_degree": rotate_degree,
        "translate": translate,
        "scale": scale,
    }
    for horizontal_flip, vertical_flips, rotate_degree, translate, scale in itertools.product(
        horizontal_flips,
        vertical_flips,
        rotate_degrees,
        translates,
        scales,
    )
]

transforms = [
    torch.nn.Sequential(
        HorizontalFlip(flip=param["horizontal_flip"]),
        VerticalFlip(flip=param["vertical_flips"]),
        Affine(degrees=param["rotate_degree"], translate=param["translate"], scale=param["scale"]),
        T.ConvertImageDtype(torch.float32),
    )
    for param in params
]

batch_size = 32 * 6

# model.to("cpu")
model.to("cuda")

y_hats, target = evaluate()

print(y_hats.shape)
print(target.shape)
# ~20:50

In [ ]:
def accuracy(y_hats, target):
    if y_hats.size(1) == 0:
        return 0.0

    y_hat = torch.mean(F.softmax(y_hats, dim=2), dim=1)
    pred = torch.argmax(y_hat, dim=1)

    acc = Accuracy(task="multiclass", num_classes=len(label_names), average="macro")
    return acc(pred, target).item()


print(accuracy(y_hats, target))  # acc_tta 0.8037662506103516 0.821164608001709
print(accuracy(y_hats[:, [0]], target))  # acc 0.7875436544418335

# Delete

In [ ]:
model_log_dir = root / "logs" / "train" / "multiruns" / "2023-11-10_08-36-16" / "0"  # graceful-plasma-2705

checkpoint = sorted(model_log_dir.glob("checkpoints/epoch_*.ckpt"))[-1]
model = FetalLitModule.load_from_checkpoint(str(checkpoint))
# disable randomness, dropout, etc...
model.eval()

model.to("cuda")

y_hats, target = evaluate()

print(accuracy(y_hats, target))  # acc_tta 0.8037662506103516 0.821164608001709
print(accuracy(y_hats[:, [0]], target))  # acc 0.7875436544418335

# ----

In [ ]:
individual_score = pd.DataFrame(params)
individual_score["accuracy"] = [accuracy(y_hats[:, [i]], target) for i in range(y_hats.size(1))]

with pd.option_context("display.max_rows", None, "display.max_columns", None):  # more options can be specified also
    print(individual_score)

In [ ]:
def gen(left, n):
    if n == 1:
        return [[v] for v in left]

    rs = gen(left, n - 1)

    rrs = []
    for v in left:
        for r in rs:
            rrs.append(r + [v])
    return rrs


def cumulative_accuracy(row):
    idxs = []
    for i, param in enumerate(params):
        add = True
        for key, value in params[0].items():
            if row[key] == "-" and param[key] != value:
                add = False

        if add:
            idxs.append(i)

    return accuracy(y_hats[:, idxs], target)


cumulative_score = pd.DataFrame(gen(["-", "+"], 5), columns=params[0].keys())
cumulative_score["accuracy"] = [cumulative_accuracy(cumulative_score.loc[i]) for i in range(len(cumulative_score))]
cumulative_score

In [ ]:
def aggregate_score(horizontal_flips, vertical_flips, rotate_degrees, translates, scales):
    aggregate_params = [
        {
            "horizontal_flip": horizontal_flip,
            "vertical_flips": vertical_flips,
            "rotate_degree": rotate_degree,
            "translate": translate,
            "scale": scale,
        }
        for horizontal_flip, vertical_flips, rotate_degree, translate, scale in itertools.product(
            horizontal_flips,
            vertical_flips,
            rotate_degrees,
            translates,
            scales,
        )
    ]

    idxs = []
    for i, param in enumerate(params):
        for agr_param in aggregate_params:
            if param == agr_param:
                idxs.append(i)

    return accuracy(y_hats[:, idxs], target)


aggregate_score(
    horizontal_flips=[False],
    # horizontal_flips=[False, True],
    vertical_flips=[False],
    # vertical_flips=[False, True],
    rotate_degrees=[0],
    # rotate_degrees=[0, -5, 5],
    # rotate_degrees=[0, -5, -10, 5, 10],
    # rotate_degrees=[0, -5, -10, -15, 5, 10, 15],
    translates=[(0.0, 0.0)],
    # translates=[(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)],
    # translates=list(itertools.product([0.0, 0.1, -0.1], [0.0, 0.1, -0.1])),
    scales=[1.0],
    # scales=[1.0, 1.05],
    # scales=[1.0, 1.05, 1.10],
    # scales=[1.0, 1.05, 1.10, 1.15],
    # scales=[1.0, 1.05, 1.10, 1.15, 1.20],
    # scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25],
    # scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30],
)

In [ ]:
def search_aggregate_score(horizontal_flips, vertical_flips, rotate_degrees, translates, scales):
    return [
        {
            "score": aggregate_score(horizontal_flip, vertical_flips, rotate_degree, translate, scale),
            "horizontal_flip": horizontal_flip,
            "vertical_flips": vertical_flips,
            "rotate_degree": rotate_degree,
            "translate": translate,
            "scale": scale,
        }
        for horizontal_flip, vertical_flips, rotate_degree, translate, scale in itertools.product(
            horizontal_flips,
            vertical_flips,
            rotate_degrees,
            translates,
            scales,
        )
    ]


scores = search_aggregate_score(
    horizontal_flips=[[False], [False, True]],
    vertical_flips=[[False], [False, True]],
    rotate_degrees=[[0], [0, -5, 5], [0, -5, -10, 5, 10]],
    translates=[[(0.0, 0.0)], [(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]],
    scales=[[1.0], [1.0, 1.05, 1.10], [1.0, 1.05, 1.10, 1.15, 1.20]],
)

print("Best score")
pprint(max(scores, key=lambda x: x["score"]))
print("")
sorted(scores, key=lambda x: x["score"], reverse=True)  # 0.8133314251899719

In [ ]:
def greed_search_aggregate_score(horizontal_flips, vertical_flips, rotate_degrees, translates, scales):
    horizontal_flips = perm(horizontal_flips)
    vertical_flips = perm(vertical_flips)
    rotate_degrees = perm(rotate_degrees)
    translates = perm(translates)
    scales = perm(scales)

    aggregate_params = [
        {
            "score": aggregate_score(horizontal_flip, vertical_flips, rotate_degree, translate, scale),
            "horizontal_flip": horizontal_flip,
            "vertical_flips": vertical_flips,
            "rotate_degree": rotate_degree,
            "translate": translate,
            "scale": scale,
        }
        for horizontal_flip, vertical_flips, rotate_degree, translate, scale in itertools.product(
            horizontal_flips,
            vertical_flips,
            rotate_degrees,
            translates,
            scales,
        )
    ]

    return max(aggregate_params, key=lambda x: x["score"])


def perm(values):
    list_of_lists = [list(itertools.combinations(values, i + 1)) for i in range(len(values))]
    return list(itertools.chain(*list_of_lists))


greed_search_aggregate_score(
    #     horizontal_flips=[False],
    horizontal_flips=[False, True],
    vertical_flips=[False],
    #     rotate_degrees=[0],
    #     rotate_degrees=[0, -5, 5],
    #     rotate_degrees=[0, -5, -10, 5, 10],
    rotate_degrees=[0, -5, -10, -15, 5, 10, 15],
    #     translates=[(0.0, 0.0)],
    #     translates=[(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)],
    translates=list(itertools.product([0.0, 0.1, -0.1], [0.0, 0.1, -0.1])),
    #     scales=[1.0],
    #     scales=[1.0, 1.05],
    #     scales=[1.0, 1.05, 1.10],
    #     scales=[1.0, 1.05, 1.10, 1.15],
    #     scales=[1.0, 1.05, 1.10, 1.15, 1.20],
    #     scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25],
    scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30],
)

# Summary

In [ ]:
# VTA?

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0]
translates = [(0.0, 0.0)]
scales = [1.0]
# 0.8037662506103516
# 0.7986749410629272
# 0.7922136783599854
# 0.7947545051574707

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0, -5, -10, 5, 10]
translates = [(0.0, 0.0)]
scales = [1.0]
# 0.8122719526290894
# 0.8002059459686279
# 0.7947515249252319
# 0.7977738380432129

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0]
translates = [(0.0, 0.0)]
scales = [1.0, 1.05, 1.10, 1.15]
# 0.8118382096290588
# 0.7972695827484131
# 0.789408266544342
# 0.7966556549072266

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0]
translates = [(0.0, 0.0)]
scales = [1.0, 1.05, 1.10, 1.15, 1.20]
# 0.814094603061676
# 0.8002622723579407
# 0.7918455600738525
# 0.7956793308258057

In [ ]:
# playful-haze-2111
# mix_ra
# model_log_dir = root / "logs" / "train" / "multiruns" / "2023-04-22_14-32-35" / "4"

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0, -5, -10, 5, 10]
translates = [(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]
scales = [1.0, 1.05, 1.10, 1.15, 1.20]

# 0.7875436544418335
# 0.8037662506103516
# 0.821164608001709


# scales=[1.0, 1.05, 1.10]
# 0.8078333735466003

# scales=[1.0, 1.05, 1.10, 1.15]
# 0.8159346580505371

# scales=[1.0, 1.05, 1.10, 1.15, 1.20]
# 0.821164608001709

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25]
#

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30]
# 0.8252905011177063

In [ ]:
# lunar-terrain-2133
# rac
# model_log_dir = root / "logs" / "train" / "multiruns" / "2023-04-23_12-16-05" / "0"

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0, -5, -10, 5, 10]
translates = [(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]
scales = [1.0, 1.05, 1.10, 1.15]

# 0.7939079403877258
# 0.7986749410629272
# 0.8068532943725586


# scales=[1.0, 1.05, 1.10]
# 0.8031753301620483

# scales=[1.0, 1.05, 1.10, 1.15]
# 0.8068532943725586

# scales=[1.0, 1.05, 1.10, 1.15, 1.20]
# 0.8048830628395081

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25]
# 0.7977616190910339

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30]
# 0.7976198792457581

In [ ]:
# dry-resonance-2119
# mix_rac_ra
# model_log_dir = root / "logs" / "train" / "multiruns" / "2023-04-22_18-04-17" / "5"

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0, -5, -10, 5, 10]
translates = [(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]
scales = [1.0, 1.05, 1.10, 1.15, 1.20]

# 0.7930771708488464
# 0.7922136783599854
# 0.7974439263343811


# scales=[1.0, 1.05, 1.10]
# 0.7962639927864075

# scales=[1.0, 1.05, 1.10, 1.15]
# 0.7965254187583923

# scales=[1.0, 1.05, 1.10, 1.15, 1.20]
# 0.7974439263343811

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25]
# 0.7974885106086731

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30]
# 0.7954345941543579

In [ ]:
# fast-shadow-2103
# mix_rac
# model_log_dir = root / "logs" / "train" / "multiruns" / "2023-04-22_11-42-40" / "3"

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0, -5, -10, -15, 5, 10, 15]
translates = [(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]
scales = [1.0]

# 0.782551646232605
# 0.7947545051574707
# 0.8007570505142212


# scales=[1.0, 1.05, 1.10]
# 0.7916216850280762

# scales=[1.0, 1.05, 1.10, 1.15]
# 0.7899205684661865

# scales=[1.0, 1.05, 1.10, 1.15, 1.20]
# 0.7916465997695923

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25]
# 0.7920457124710083

# scales=[1.0, 1.05, 1.10, 1.15, 1.20, 1.25, 1.30]
# 0.7914426326751709

In [ ]:
# faithful-jazz-2130
# mix_ra
# model_log_dir = root / "logs" / "train" / "multiruns" / "2023-04-23_01-59-06" / "4"

horizontal_flips = [False, True]
vertical_flips = [False]
rotate_degrees = [0, -5, -10, 5, 10]
translates = [(0.0, 0.0)]
# translates=[(0.0, 0.0), (0.1, 0.1), (-0.1, 0.1), (-0.1, -0.1), (0.1, -0.1)]
scales = [1.0, 1.05, 1.10, 1.15, 1.20]

# 0.7793711423873901
# 0.792731761932373
#